In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table
pd.set_option('display.max_rows', None,'display.max_columns', None)

2022-04-20 23:18:56.450715: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/count
2022-04-20 23:18:56.450739: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/count
2022-04-20 23:18:56.450745: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/api
2022-04-20 23:18:56.450748: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/api


#### Time Series Prediction

In [2]:
# NYSE = pd.read_csv('/Users/lei/Desktop/Lecture_Notes/McGill_Courses/James_2021_Rscripts/Data/NYSE.csv', index_col='Unnamed: 0')
NYSE = pd.read_csv('/Users/lei/Desktop/Lecture_Notes/McGill_Courses/James_2021_Rscripts/Data/NYSE.csv')
print(NYSE.columns)
NYSE.isnull().sum()

Index(['Unnamed: 0', 'date', 'day_of_week', 'DJ_return', 'log_volume',
       'log_volatility', 'train'],
      dtype='object')


Unnamed: 0        0
date              0
day_of_week       0
DJ_return         0
log_volume        0
log_volatility    0
train             0
dtype: int64

In [3]:
NYSE.head()

,Unnamed: 0,date,day_of_week,DJ_return,log_volume,log_volatility,train
0,1,1962-12-03,mon,-0.004461,0.032573,-13.127403,True
1,2,1962-12-04,tues,0.007813,0.346202,-11.749305,True
2,3,1962-12-05,wed,0.003845,0.525306,-11.665609,True
3,4,1962-12-06,thur,-0.003462,0.210182,-11.626772,True
4,5,1962-12-07,fri,0.000568,0.044187,-11.728130,True


In [4]:
train_dates = NYSE.date <= '1980-01-02'
fig = make_subplots(rows=3, cols=1, subplot_titles=('DJ_return', 'log_volume', 'log_volatility'))

fig.add_trace(go.Scatter(x=NYSE.date.loc[train_dates], y=NYSE.DJ_return.loc[train_dates], name='train', line=dict(color='black', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=NYSE.date.loc[~train_dates], y=NYSE.DJ_return.loc[~train_dates], name='test', line=dict(color='grey', width=1)), row=1, col=1)

fig.add_trace(go.Scatter(x=NYSE.date.loc[train_dates], y=NYSE.log_volume.loc[train_dates], name='train', line=dict(color='black', width=1)), row=2, col=1)
fig.add_trace(go.Scatter(x=NYSE.date.loc[~train_dates], y=NYSE.log_volume.loc[~train_dates], name='test', line=dict(color='grey', width=1)), row=2, col=1)

fig.add_trace(go.Scatter(x=NYSE.date.loc[train_dates], y=NYSE.log_volatility.loc[train_dates], name='train', line=dict(color='black', width=1)), row=3, col=1)
fig.add_trace(go.Scatter(x=NYSE.date.loc[~train_dates], y=NYSE.log_volatility.loc[~train_dates], name='test', line=dict(color='grey', width=1)), row=3, col=1)
fig.show()

In [5]:
xdata = NYSE[['DJ_return', 'log_volume', 'log_volatility']]
istrain = NYSE.train
xdata.head()

,DJ_return,log_volume,log_volatility
0,-0.004461,0.032573,-13.127403
1,0.007813,0.346202,-11.749305
2,0.003845,0.525306,-11.665609
3,-0.003462,0.210182,-11.626772
4,0.000568,0.044187,-11.728130


In [ ]:
# standardize each of the variables
from sklearn.preprocessing import scale
xdata = pd.DataFrame(data=scale(xdata), columns=xdata.columns)
xdata.head()

,DJ_return,log_volume,log_volatility
0,-0.549823,0.175075,-4.357078
1,0.905200,1.517291,-2.529058
2,0.434813,2.283789,-2.418037
3,-0.431397,0.935176,-2.366521
4,0.046340,0.224779,-2.500970


In [ ]:
# create lagged versions of the three time series
def lagm(x, k=1):
    nrow, ncol = x.shape
    pad = np.asarray([[np.nan] * 3] * k)
    x_trunc = xdata.iloc[: nrow-k].to_numpy()

    # inserts k rows of NA at the top, and truncates the bottom
    return pd.DataFrame(data=np.vstack((pad, x_trunc)), columns=x.columns)
    
arframe = xdata.log_volume
for i in range(5):
    arframe = pd.concat([arframe, lagm(xdata , i+1)], axis=1)

arframe.columns = ["log_volume", "L1.DJ_return", "L1.log_volume","L1.log_volatility","L2.DJ_return", "L2.log_volume", "L2.log_volatility", 
"L3.DJ_return", "L3.log_volume","L3.log_volatility","L4.DJ_return", "L4.log_volume", "L4.log_volatility", "L5.DJ_return", "L5.log_volume", "L5.log_volatility"]

arframe.head(8)

,log_volume,L1.DJ_return,L1.log_volume,L1.log_volatility,L2.DJ_return,L2.log_volume,L2.log_volatility,L3.DJ_return,L3.log_volume,L3.log_volatility,L4.DJ_return,L4.log_volume,L4.log_volatility,L5.DJ_return,L5.log_volume,L5.log_volatility
0,0.175075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.517291,-0.549823,0.175075,-4.357078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.283789,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.935176,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078,NaN,NaN,NaN,NaN,NaN,NaN
4,0.224779,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078,NaN,NaN,NaN
5,0.605918,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078
6,-0.013661,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058
7,0.042552,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037


In [ ]:
arframed = pd.concat([arframe, NYSE.day_of_week], axis=1)
arframed.head()

,log_volume,L1.DJ_return,L1.log_volume,L1.log_volatility,L2.DJ_return,L2.log_volume,L2.log_volatility,L3.DJ_return,L3.log_volume,L3.log_volatility,L4.DJ_return,L4.log_volume,L4.log_volatility,L5.DJ_return,L5.log_volume,L5.log_volatility,day_of_week
0,0.175075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mon
1,1.517291,-0.549823,0.175075,-4.357078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tues
2,2.283789,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wed
3,0.935176,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078,NaN,NaN,NaN,NaN,NaN,NaN,thur
4,0.224779,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078,NaN,NaN,NaN,fri


In [ ]:
# remove these rows, and adjust istrain accordingly
ar_df = arframe.iloc[5:]
ard_df = arframed.iloc[5:]
print(ar_df.shape, ard_df.shape)
ar_df.head()


(6046, 16) (6046, 17)


,log_volume,L1.DJ_return,L1.log_volume,L1.log_volatility,L2.DJ_return,L2.log_volume,L2.log_volatility,L3.DJ_return,L3.log_volume,L3.log_volatility,L4.DJ_return,L4.log_volume,L4.log_volatility,L5.DJ_return,L5.log_volume,L5.log_volatility
5,0.605918,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078
6,-0.013661,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058
7,0.042552,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037
8,-0.419836,0.377081,0.042552,-1.551515,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521
9,-0.556065,-0.411718,-0.419836,-1.597607,0.377081,0.042552,-1.551515,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970


In [ ]:
from sklearn import linear_model

# fit the linear AR model to the training data
# print(ar_df[istrain[5:]].shape)
X_train = ar_df.loc[istrain[5:], ar_df.columns != 'log_volume'] 
y_train = ar_df.loc[istrain[5:], ar_df.columns == 'log_volume']
X_test = ar_df.loc[~istrain[5:], ar_df.columns != 'log_volume']
y_test = ar_df.loc[~istrain[5:], ar_df.columns == 'log_volume']

lm = linear_model.LinearRegression().fit(X=X_train, y=y_train)
y_hat = lm.predict(X=X_test)

def R2_test(y_hat, y_test):
    return 1 - float(((y_hat - y_test)**2).mean()) / float(y_test.var())
R2_test(y_hat, y_test)

0.41322299369023174

In [ ]:
ard_df_day = pd.get_dummies(ard_df.day_of_week, prefix='day')
print(ard_df_day.shape)
ard_df_encode = pd.concat([ard_df.loc[:, ard_df.columns != 'day_of_week'], ard_df_day], axis=1)
ard_df_encode.head()

(6046, 5)


,log_volume,L1.DJ_return,L1.log_volume,L1.log_volatility,L2.DJ_return,L2.log_volume,L2.log_volatility,L3.DJ_return,L3.log_volume,L3.log_volatility,L4.DJ_return,L4.log_volume,L4.log_volatility,L5.DJ_return,L5.log_volume,L5.log_volatility,day_fri,day_mon,day_thur,day_tues,day_wed
5,0.605918,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078,0,1,0,0,0
6,-0.013661,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,0,0,0,1,0
7,0.042552,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0,0,0,0,1
8,-0.419836,0.377081,0.042552,-1.551515,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0,0,1,0,0
9,-0.556065,-0.411718,-0.419836,-1.597607,0.377081,0.042552,-1.551515,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,1,0,0,0,0


In [ ]:
# refit this model, including the factor variable day of week
X_train = ard_df_encode.loc[istrain[5:], ard_df_encode.columns != 'log_volume']
y_train = ard_df_encode.loc[istrain[5:], ard_df_encode.columns == 'log_volume']
X_test = ard_df_encode.loc[~istrain[5:], ard_df_encode.columns != 'log_volume']
y_test = ard_df_encode.loc[~istrain[5:], ard_df_encode.columns == 'log_volume']

lm = linear_model.LinearRegression().fit(X=X_train, y=y_train)
y_hat = lm.predict(X=X_test)

R2_test(y_hat, y_test)

0.4598616487215391

##### Fit an RNN model

In [ ]:
xrnn = ar_df.loc[:, ar_df.columns != 'log_volume']
xrnn.head()

,L1.DJ_return,L1.log_volume,L1.log_volatility,L2.DJ_return,L2.log_volume,L2.log_volatility,L3.DJ_return,L3.log_volume,L3.log_volatility,L4.DJ_return,L4.log_volume,L4.log_volatility,L5.DJ_return,L5.log_volume,L5.log_volatility
5,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058,-0.549823,0.175075,-4.357078
6,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037,0.905200,1.517291,-2.529058
7,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521,0.434813,2.283789,-2.418037
8,0.377081,0.042552,-1.551515,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970,-0.431397,0.935176,-2.366521
9,-0.411718,-0.419836,-1.597607,0.377081,0.042552,-1.551515,-0.006294,-0.013661,-1.505667,-1.304126,0.605918,-1.366028,0.046340,0.224779,-2.500970


In [ ]:
# xrnn.to_numpy().reshape(xrnn.shape[0], 3, 5)
# print(np.flip(xrnn.to_numpy().reshape(xrnn.shape[0], 3, 5), axis=2).shape)
# np.swapaxes(np.flip(xrnn.to_numpy().reshape(xrnn.shape[0], 3, 5), axis=2), axis1=1, axis2=2).shape
X = np.swapaxes(np.flip(xrnn.to_numpy().reshape(xrnn.shape[0], 3, 5), axis=2), axis1=1, axis2=2)
print(X.shape)
X_train = X[istrain[5:], :, :]
y_train = ar_df.loc[istrain[5:], ar_df.columns == 'log_volume']
X_test = X[~istrain[5:], :, :]
y_test = ar_df.loc[~istrain[5:], ar_df.columns == 'log_volume'] 


(6046, 5, 3)


In [ ]:
# Define Sequential model 
model = keras.Sequential(
    [
        layers.SimpleRNN(12, input_shape=(5,3), dropout=.1, recurrent_dropout=.1),  
        layers.Dense(1), # output layer
    ]
)

model.compile(loss='mse', optimizer='rmsprop')

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 12)                192       
_________________________________________________________________
dense (Dense)                (None, 1)                 13        
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


2022-04-20 22:14:41.003437: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
history = model.fit(x=X_train, y=y_train, batch_size=64, epochs=200, validation_data=(X_test,y_test))

2022-04-20 22:14:41.449669: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:179] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
67/67 [==============================] - 1s 4ms/step - loss: 1.0432 - val_loss: 0.9021
Epoch 2/200
67/67 [==============================] - 0s 2ms/step - loss: 0.6864 - val_loss: 0.7794
Epoch 3/200
67/67 [==============================] - 0s 2ms/step - loss: 0.5945 - val_loss: 0.7387
Epoch 4/200
67/67 [==============================] - 0s 2ms/step - loss: 0.5688 - val_loss: 0.7196
Epoch 5/200
67/67 [==============================] - 0s 2ms/step - loss: 0.5438 - val_loss: 0.7121
Epoch 6/200
67/67 [==============================] - 0s 2ms/step - loss: 0.5371 - val_loss: 0.7077
Epoch 7/200
67/67 [==============================] - 0s 2ms/step - loss: 0.5314 - val_loss: 0.6985
Epoch 8/200
67/67 [==============================] - 0s 2ms/step - loss: 0.5261 - val_loss: 0.6994
Epoch 9/200
67/67 [==============================] - 0s 2ms/step - loss: 0.5172 - val_loss: 0.6884
Epoch 10/200
67/67 [==============================] - 0s 2ms/step - loss: 0.5160 - val_loss: 0.6897
Epoch 11/

In [ ]:
print(y_test.var(), y_train.var())
y_hat = model.predict(x=X_test)
print(R2_test(y_hat, y_test))


log_volume    1.054282
dtype: float64 log_volume    0.976095
dtype: float64
0.40481458703118933


In [ ]:
# model.history.history.keys()
res = pd.DataFrame.from_dict(history.history)
res.head()

fig = make_subplots(rows=1, cols=2, subplot_titles=('Loss', 'MAE'))

# fig = go.Figure()
fig.add_trace(go.Scatter(x=history.epoch, y=res.loss, name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(x=history.epoch, y=res.val_loss, name='val_loss'), row=1, col=1)

# fig.add_trace(go.Scatter(x=history.epoch, y=res.mae, name='mae'), row=1, col=2)
# fig.add_trace(go.Scatter(x=history.epoch, y=res.val_mae, name='val_mae'), row=1, col=2)

fig.show()

In [ ]:
X_train = ard_df_encode.loc[istrain[5:], ard_df_encode.columns != 'log_volume']
y_train = ard_df_encode.loc[istrain[5:], ard_df_encode.columns == 'log_volume']
X_test = ard_df_encode.loc[~istrain[5:], ard_df_encode.columns != 'log_volume']
y_test = ard_df_encode.loc[~istrain[5:], ard_df_encode.columns == 'log_volume']

# Define Sequential model 
arnnd = keras.Sequential(
    [
        layers.Dense(32, activation='relu', input_shape=X_train.shape),  
        layers.Dropout(.5),
        layers.Dense(1), # output layer
    ]
)

arnnd.compile(loss='mse', optimizer='rmsprop')

arnnd.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4276, 32)          672       
_________________________________________________________________
dropout (Dropout)            (None, 4276, 32)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 4276, 1)           33        
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = arnnd.fit(x=X_train, y=y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
134/134 [==============================] - 1s 3ms/step - loss: 1.5922 - val_loss: 0.8369
Epoch 2/100
134/134 [==============================] - 0s 662us/step - loss: 0.7648 - val_loss: 0.6791
Epoch 3/100
134/134 [==============================] - 0s 668us/step - loss: 0.5955 - val_loss: 0.6286
Epoch 4/100
134/134 [==============================] - 0s 671us/step - loss: 0.5384 - val_loss: 0.6071
Epoch 5/100
134/134 [==============================] - 0s 675us/step - loss: 0.5031 - val_loss: 0.5938
Epoch 6/100
134/134 [==============================] - 0s 689us/step - loss: 0.4905 - val_loss: 0.5849
Epoch 7/100
134/134 [==============================] - 0s 682us/step - loss: 0.4826 - val_loss: 0.5818
Epoch 8/100
134/134 [==============================] - 0s 684us/step - loss: 0.4588 - val_loss: 0.5781
Epoch 9/100
134/134 [==============================] - 0s 681us/step - loss: 0.4656 - val_loss: 0.5727
Epoch 10/100
134/134 [==============================] - 0s 791us/step - los

In [ ]:
res = pd.DataFrame.from_dict(history.history)
res.head()

fig = go.Figure()
fig.add_trace(go.Scatter(x=history.epoch, y=res.loss, name='loss'))
fig.add_trace(go.Scatter(x=history.epoch, y=res.val_loss, name='val_loss'))
fig.update_layout(title='Evolution of training and validation errors', xaxis_title='Epochs', yaxis_title='Loss')

fig.show()

0.4680118141881281

In [ ]:
print(NYSE.shape, y_hat.shape, NYSE.date.loc[~train_dates].shape)
# y_hat_scale = (y_hat - y_hat.mean())/(y_hat.std())

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(y_hat)
y_hat_scale = scaler.transform(y_hat) + (NYSE.log_volume.loc[~train_dates].mean() - scaler.transform(y_hat).mean())

(6051, 7) (1770, 1) (1769,)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=NYSE.date.loc[~train_dates], y=NYSE.log_volume.loc[~train_dates], name='test', line=dict(color='black', width=1)))
fig.add_trace(go.Scatter(x=NYSE.date.loc[~train_dates], y=np.append(y_hat_scale, np.nan), name='forecasted', line=dict(color='red', width=1)))
fig.update_layout(title='RNN forecaseted Trading Volume', xaxis_title='Date', yaxis_title='Trading Volume')
fig.show()

y_hat = arnnd.predict(x=X_test)
R2_test(y_hat, y_test)

0.4680118141881281